In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils
import statistics

_, input_df = dir_utils.get_data(combined=True)

In [2]:
def setup():
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
    purchases = set()
    
    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            
            if t[constants.TYPE] == 'Purchase' and ptr_utils.get_year(t[constants.TDATE]) <= 2020: 
                purchases.add(core)

    return purchases

p = setup()

In [3]:
problem_tickers = {}

def match(increment_weeks):
    res = []
    i = 0 
    for (tdate_purch, name_purch, ticker_purch)  in p:
        i+=1 
        
        if ticker_purch not in problem_tickers: 
            tdate_sale = profit.add_to_date(tdate_purch, increment_weeks)
            x = profit.share_diff(ticker_purch, tdate_sale, tdate_purch)
            if x is not None: 
                res.append([name_purch, ticker_purch, tdate_purch, tdate_sale, x])
            else:
                problem_tickers[ticker_purch] = ""
            
        print(i)
        if i % 50 == 0: 
            break
            
    return res

def get_average(res):
    l = []
    for (name_purch, ticker_purch, tdate_purch, tdate_sale, x) in res:
        l.append(x)
    return statistics.mean(l) 

def get_people_average(res):
    people = {}
    for (name_purch, ticker_purch, tdate_purch, tdate_sale, x) in res:
        people = dict_utils.increment_list_in_dictionary(people, name_purch, x)
    
    for person, profit_list in people.items():
        people[person] = round(statistics.mean(profit_list), 4)
    
    return people 

def get_ticker_average(res):
    tickers = {}
    for (name_purch, ticker_purch, tdate_purch, tdate_sale, x) in res:
        tickers = dict_utils.increment_list_in_dictionary(tickers, ticker_purch, x)
    
    for ticker, profit_list in tickers.items():
        tickers[ticker] = round(statistics.mean(profit_list), 4) 
    
    return tickers

In [4]:
one_week = match(1)
print(get_average(one_week))
print(get_people_average(one_week))
print(get_ticker_average(one_week))

# two_weeks = match(2)
# get_average(two_weeks)
# get_people_average(two_weeks)
# get_ticker_average(two_weeks)

# month = match(4)
# get_average(month)
# get_people_average(month)
# get_ticker_average(month)

# two_months = match(8)
# get_average(two_months)
# get_people_average(two_months)
# get_ticker_average(two_months)

# four_months = match(16)
# get_average(four_months)
# get_people_average(four_months)
# get_ticker_average(four_months)

# eight_months = match(32)
# get_average(eight_months)
# get_people_average(eight_months)
# get_ticker_average(eight_months)

# year = match(52)
# get_average(year)
# get_people_average(year)
# get_ticker_average(year)

# two_years = match(104)
# get_average(two_years)
# get_people_average(two_years)
# get_ticker_average(two_years)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
# week = [name, ticker, tdate_purch, tdate_sale, profit]

x = [(one_week, "week"),
     (two_weeks, "two_weeks"),
    (month, "month"),
    (two_months, "twomonths"),
    (four_months, "fourmonths"),
    (eight_months, "eightmonths"),
    (year, "year"),
    (two_years, "twoyears")]

print(len(week))
print(len(month))
print(len(twomonths))
print(len(fourmonths))
print(len(eightmonths))
print(len(year))

# official_name, ticker, tdate_purch : [ (tdate_sale : diff) , (tdate_sale : diff), (tdate_sale : diff), ...]
sell_profit = {}
for l, n in x:
    for (official_name, ticker, tdate_purch, tdate_sale, diff) in l:
        sell_profit = dict_utils.increment_list_in_dictionary(sell_profit, (official_name, ticker, tdate_purch), (tdate_sale, diff))

l = []
for (name, ticker, tdate_purch), v in sell_profit.items():
    r = [name, ticker, tdate_purch]
    for (x,y) in v:
        r.append(x)
        r.append(y)
    l.append(r)
    
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT)
csv_utils.make_csv_base(dir, "combinedadsatt", [constants.OFFICIAL, constants.TICKER, constants.BUYDATE, "week", "profit_week", "month", "profit_month", "twomonths", "profit_twomonths", "fourmonths", "profit_fourmonths", "eightmonths", "profit_eightmonths", "year", "profit_year"], l)